这里说频率是指一个机场航班出现的频率

## 加载数据

数据已经预处理过了

In [1]:
import pandas as pd


# 加载数据
pd.set_option('display.max_columns', None)  # 显示所有列

data = pd.read_csv('../result/pre_2023-2025_with_comp_test.csv', dtype={'aircraft': str})

# 查看前几行数据，确保加载成功
print(data.head())

print(data.info())

   flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0  3c6705   94.0      190     1       1      1.33   45  a9f275  24cc4b   
1  284078  162.0      322     1       1      2.10  143  e3d141  d618e3   
2  484808  169.0      738     1       1      4.52  154  9550bc  2e1ea1   
3  0f40c9  167.0      320     1       1      4.28  167  9550bc  2e1ea1   
4  ce2225  166.0      320     1       1      1.33  181  9550bc  24cc4b   

        c   unit_price  competitor_price  year  month  day  weekday  hour  \
0  7f9c2b   481.555556       -119.744444  2024      7    1        0    12   
1  7f9c2b   902.167832          0.000000  2024      7    1        0    10   
2  7f9c2b  1540.357143        283.690476  2024      7    1        0    14   
3  7f9c2b  1345.449102        960.619556  2024      7    1        0    21   
4  7f9c2b   593.480663        -54.049640  2024      7    1        0    23   

   minute from   to  
0      30  a9f  c4b  
1      30  e3d  8e3  
2      15  955  ea1  
3   

In [2]:
# # 随机抽取5万条数据
# data = data.sample(n=20000, random_state=42)

# 查看抽样后的数据信息
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333939 entries, 0 to 333938
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   flt_no            333939 non-null  object 
 1   cap               333939 non-null  float64
 2   aircraft          333939 non-null  object 
 3   legs              333939 non-null  int64  
 4   leg_no            333939 non-null  int64  
 5   duration          333907 non-null  float64
 6   pax               333939 non-null  int64  
 7   a                 333939 non-null  object 
 8   b                 333939 non-null  object 
 9   c                 333939 non-null  object 
 10  unit_price        333939 non-null  float64
 11  competitor_price  333939 non-null  float64
 12  year              333939 non-null  int64  
 13  month             333939 non-null  int64  
 14  day               333939 non-null  int64  
 15  weekday           333939 non-null  int64  
 16  hour              33

In [3]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    333939.000000
mean        123.203549
std          53.691957
min          20.000000
25%          84.000000
50%         131.000000
75%         160.000000
max         301.000000
Name: pax, dtype: float64



## 编码分类变量

### 新增城市标签

In [4]:
import json
# 加载字典
with open('../result/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'000': 2, '13f': 1, '8e3': 1, 'a18': 1, 'eda': 2, '01a': 2, 'b5a': 1, 'ea1': 1, '03f': 2, '1be': 1, '3bc': 1, '578': 1, '5df': 1, '5fb': 1, '760': 1, '792': 1, '7ff': 1, '90a': 1, '974': 1, 'bbb': 1, 'cb5': 1, 'ffe': 1, '059': 2, '022': 1, '077': 1, '0d7': 1, '0dd': 1, '1a3': 1, '1e9': 1, '1ed': 1, '25c': 1, '311': 1, '354': 1, '451': 1, '510': 1, '5d7': 2, '63a': 1, '666': 0, '6e3': 0, '7f1': 1, '812': 1, '8a1': 1, '9d4': 2, '9e9': 1, 'b3a': 1, 'bff': 1, 'c24': 1, 'c3a': 1, 'c4c': 1, 'c98': 1, 'c9f': 1, 'cb7': 1, 'd7c': 1, 'e76': 1, 'e9c': 1, 'efb': 1, '05c': 2, '141': 1, '157': 1, '709': 1, '8ac': 0, '97f': 1, '988': 1, 'a2f': 1, 'b09': 1, 'c14': 1, 'c4b': 0, 'e5b': 1, 'ec2': 1, '074': 2, '4d0': 1, '695': 1, 'c2c': 1, 'fa9': 1, '08c': 2, '08f': 2, '3c5': 1, '5e8': 1, 'a72': 1, 'd54': 1, 'ff1': 1, '0ac': 2, '0c8': 2, '0ca': 0, 'e09': 0, '0d1': 2, '13e': 1, '1ee': 1, '43e': 1, '0db': 2, '919': 1, 'd08': 1, '0ff': 2, '11b': 0, '123': 2, '5de': 1, '8eb': 1, '124': 2, '130': 2, '1

In [5]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [6]:
import json

# 加载字典
with open('../result/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'000': [-1.3355571031570435, -0.7252184152603149], '13f': [0.7116603255271912, -0.38071975111961365], '8e3': [0.6560439467430115, -0.48669493198394775], 'a18': [0.2972618341445923, -0.32849031686782837], 'eda': [-0.3947368860244751, -1.0661070346832275], '01a': [-1.0750128030776978, -0.22083736956119537], 'b5a': [0.8525970578193665, -0.6555958390235901], 'ea1': [0.22486557066440582, 0.1256379634141922], '03f': [-0.5686646699905396, -0.8201126456260681], '1be': [0.9253679513931274, -0.583831250667572], '3bc': [1.186996579170227, -0.8084284067153931], '578': [1.038602352142334, -0.6717441082000732], '5df': [0.9066036939620972, -0.8123182654380798], '5fb': [0.9960507154464722, -0.22026796638965607], '760': [1.0199142694473267, -0.6170554161071777], '792': [0.242161363363266, -0.4239804744720459], '7ff': [0.9492039084434509, -0.339489221572876], '90a': [0.010870625264942646, -0.008459429256618023], '974': [1.2856382131576538, -0.6693836450576782], 'bbb': [0.688012421131134, -0.6741

In [7]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])

        a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0                 NaN            NaN            NaN            NaN   
1                 NaN            NaN            NaN            NaN   
2                 NaN            NaN            NaN            NaN   
3                 NaN            NaN            NaN            NaN   
4                 NaN            NaN            NaN            NaN   
...               ...            ...            ...            ...   
333934            NaN            NaN            NaN            NaN   
333935            NaN            NaN            NaN            NaN   
333936            NaN            NaN            NaN            NaN   
333937            NaN            NaN            NaN            NaN   
333938            NaN            NaN            NaN            NaN   

        c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                 NaN            NaN         -1.308495          1.850374   
1      

### 统计不同城市的频率

### 对'a', 'b', 'c', 'from', 'to'进行频率编码

In [8]:
# 加载city_map
with open('../result/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

        flt_no    cap aircraft  legs  leg_no  duration  pax   a   b   c  \
0       3c6705   94.0      190     1       1      1.33   45 NaN NaN NaN   
1       284078  162.0      322     1       1      2.10  143 NaN NaN NaN   
2       484808  169.0      738     1       1      4.52  154 NaN NaN NaN   
3       0f40c9  167.0      320     1       1      4.28  167 NaN NaN NaN   
4       ce2225  166.0      320     1       1      1.33  181 NaN NaN NaN   
...        ...    ...      ...   ...     ...       ...  ...  ..  ..  ..   
333934  4cf718  167.0      32C     3       3      3.35   29 NaN NaN NaN   
333935  4cf718  167.0      32C     3       3      3.35   29 NaN NaN NaN   
333936  5951a4  173.0      73Z     1       1      2.57  166 NaN NaN NaN   
333937  408f86   95.0      190     3       2      1.70   85 NaN NaN NaN   
333938  b60fa0   95.0      190     3       2      2.25   67 NaN NaN NaN   

         unit_price  competitor_price  year  month  day  weekday  hour  \
0        481.555556      

### 对'flt_no', 'bd_type', 'aircraft'进行标签编码

In [9]:
data.head(5)

,flt_no,cap,aircraft,legs,leg_no,duration,pax,a,b,c,unit_price,competitor_price,year,month,day,weekday,hour,minute,from,to,a_label,b_label,c_label,from_label,to_label,a_embedding_1,a_embedding_2,b_embedding_1,b_embedding_2,c_embedding_1,c_embedding_2,from_embedding_1,from_embedding_2,to_embedding_1,to_embedding_2
0,3c6705,94.0,190,1,1,1.33,45,NaN,NaN,NaN,481.555556,-119.744444,2024,7,1,0,12,30,1690,49986,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.308495,1.850374,0.017864,1.593143
1,284078,162.0,322,1,1,2.10,143,NaN,NaN,NaN,902.167832,0.000000,2024,7,1,0,10,30,562,47042,NaN,NaN,NaN,2,1,NaN,NaN,NaN,NaN,NaN,NaN,-0.670601,-0.307947,0.656044,-0.486695
2,484808,169.0,738,1,1,4.52,154,NaN,NaN,NaN,1540.357143,283.690476,2024,7,1,0,14,15,3852,110103,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,-1.591784,1.228054,0.224866,0.125638
3,0f40c9,167.0,320,1,1,4.28,167,NaN,NaN,NaN,1345.449102,960.619556,2024,7,1,0,21,30,3852,110103,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN,NaN,-1.591784,1.228054,0.224866,0.125638
4,ce2225,166.0,320,1,1,1.33,181,NaN,NaN,NaN,593.480663,-54.049640,2024,7,1,0,23,15,3852,49986,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.591784,1.228054,0.017864,1.593143


In [10]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no',  'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('../result/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1104         0
1     721        11
2    1310        49
3     278         9
4    3611         9
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    1104         0
1     721        11
2    1310        49
3     278         9
4    3611         9


## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [11]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','competitor_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

In [12]:
test_x = X.head(1)
test_x

,flt_no,cap,aircraft,legs,leg_no,duration,a,b,c,year,month,day,weekday,hour,minute,from,to,unit_price,competitor_price,a_label,b_label,c_label,from_label,to_label,a_embedding_1,a_embedding_2,b_embedding_1,b_embedding_2,c_embedding_1,c_embedding_2,from_embedding_1,from_embedding_2,to_embedding_1,to_embedding_2
0,1104,94.0,0,1,1,1.33,NaN,NaN,NaN,2024,7,1,0,12,30,1690,49986,481.555556,-119.744444,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,-1.308495,1.850374,0.017864,1.593143


In [13]:
y.head(5)

0     45
1    143
2    154
3    167
4    181
Name: pax, dtype: int64

### 对x进行标准化

In [14]:
import pandas as pd
import numpy as np
import joblib


# 1️⃣ 加载训练时保存的标准化器
scaler = joblib.load('../result/encoder/standard_scaler_x.pkl')
print("✅ 已加载标准化器 standard_scaler_x.pkl")

# 2️⃣ 找出当前数据中没有 NaN 的列
valid_cols = [col for col in X.columns if not X[col].isnull().any()]
nan_cols = [col for col in X.columns if X[col].isnull().any()]

print(f"🧩 可标准化列数: {len(valid_cols)}")
print(f"🚫 含 NaN 列数: {len(nan_cols)}")

# 3️⃣ 创建副本，防止修改原始数据
data_scaled = X.copy()

# 4️⃣ 仅对无 NaN 的列进行标准化（保持列顺序）
# ⚠️ 注意：如果 valid_cols 中有训练时未见过的列，会触发 shape mismatch。
#          因此建议 valid_cols 是训练阶段列的子集。
try:
    data_scaled[valid_cols] = scaler.transform(data[valid_cols])
except ValueError as e:
    print("❌ 标准化时列不匹配：", e)
    print("⚠️ 请确保 valid_cols 与训练时标准化的列一致或为其子集。")

data_scaled = pd.DataFrame(data_scaled, index=data.index)



# 5️⃣ 查看结果
pd.set_option('display.max_columns', None)
print("\n✅ 标准化后的数据预览（前1行）：")
print(data_scaled.head(1))

# （可选）查看未标准化列
if nan_cols:
    print("\n⚠️ 以下列未参与标准化（含 NaN 值，保持原样）：")
    print(nan_cols)

# （可选）保存标准化后的结果
# data_scaled.to_csv("new_data_scaled.csv", index=False)


✅ 已加载标准化器 standard_scaler_x.pkl
🧩 可标准化列数: 21


🚫 含 NaN 列数: 13

✅ 标准化后的数据预览（前1行）：
     flt_no       cap  aircraft      legs    leg_no  duration   a   b   c  \
0 -0.843377 -2.007241 -1.461885 -1.046281 -0.624097      1.33 NaN NaN NaN   

      year     month       day   weekday      hour   minute      from  \
0  0.26846  0.468578 -1.684848 -1.509648 -0.343375  0.20606 -1.292931   

         to  unit_price  competitor_price  a_label  b_label  c_label  \
0  0.139294   -0.552834         -0.545943      NaN      NaN      NaN   

   from_label  to_label  a_embedding_1  a_embedding_2  b_embedding_1  \
0    -2.82258 -2.813335            NaN            NaN            NaN   

   b_embedding_2  c_embedding_1  c_embedding_2  from_embedding_1  \
0            NaN            NaN            NaN         -1.593931   

   from_embedding_2  to_embedding_1  to_embedding_2  
0          3.200172       -1.424629        3.099522  

⚠️ 以下列未参与标准化（含 NaN 值，保持原样）：
['duration', 'a', 'b', 'c', 'a_label', 'b_label', 'c_label', 'a_embedding_1', 'a_embedding_2', 'b_e

### 对y进行标准化

In [15]:
# 加载y的标准化器
scaler_y = joblib.load('../result/encoder/standard_scaler_y.pkl')

# 对目标变量进行标准化
y = data['pax']  # 假设目标列名为'pax'
# y_scaled = scaler_y.transform(y.values.reshape(-1, 1))  # 将y转换为2D数组进行标准化

# # 将标准化后的数据转回DataFrame格式
# y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'], index=data.index)

# # 将标准化后的值添加到原始数据框中
# data['pax'] = y_scaled['pax_scaled']

## XGBoost模型预测

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 加载模型
model = xgb.Booster()
model.load_model("../result/model/guiyihua_xgboost_model_1000.json")

# 转换为DMatrix格式
dmatrix = xgb.DMatrix(data_scaled)

# 进行预测
scaled_predictions = model.predict(dmatrix)

# 将标准化的预测值转换回原始尺度
predictions = scaler_y.inverse_transform(scaled_predictions.reshape(-1, 1))

In [17]:
predictions

array([[ 62.29895],
       [141.13779],
       [170.3053 ],
       ...,
       [155.59047],
       [ 74.91217],
       [ 68.93506]], dtype=float32)

In [18]:
# 正确的写法
test_results = list(zip(predictions[:100], y[:100]))  # 真实值和预测值

print("\n20条测试结果（真实值 vs 预测值）:")
for i, (true_value, pred_value) in enumerate(test_results[:40]):
    # 如果是多维数组，使用 .item() 转换为标量
    true_value = true_value.item() if isinstance(true_value, np.ndarray) else true_value
    pred_value = pred_value.item() if isinstance(pred_value, np.ndarray) else pred_value
    print(f"第{i+1}条: 真实值={true_value}, 预测值={pred_value:.2f}")



20条测试结果（真实值 vs 预测值）:
第1条: 真实值=62.2989501953125, 预测值=45.00
第2条: 真实值=141.13778686523438, 预测值=143.00
第3条: 真实值=170.3052978515625, 预测值=154.00
第4条: 真实值=169.6882781982422, 预测值=167.00
第5条: 真实值=127.72280883789062, 预测值=181.00
第6条: 真实值=105.08976745605469, 预测值=138.00
第7条: 真实值=87.3626708984375, 预测值=137.00
第8条: 真实值=41.84259033203125, 预测值=50.00
第9条: 真实值=107.81560516357422, 预测值=136.00
第10条: 真实值=76.76716613769531, 预测值=99.00
第11条: 真实值=64.44551849365234, 预测值=70.00
第12条: 真实值=99.9090576171875, 预测值=62.00
第13条: 真实值=59.9420166015625, 预测值=57.00
第14条: 真实值=61.84046936035156, 预测值=67.00
第15条: 真实值=156.75875854492188, 预测值=167.00
第16条: 真实值=51.45988464355469, 预测值=30.00
第17条: 真实值=88.60279083251953, 预测值=102.00
第18条: 真实值=109.81837463378906, 预测值=163.00
第19条: 真实值=62.51297378540039, 预测值=42.00
第20条: 真实值=85.03683471679688, 预测值=119.00
第21条: 真实值=42.90611267089844, 预测值=24.00
第22条: 真实值=156.70254516601562, 预测值=161.00
第23条: 真实值=165.07235717773438, 预测值=162.00
第24条: 真实值=168.9764862060547, 预测值=170.00
第25条: 真实值=176.76637268066406, 预测值

## xgboost测试指标


In [30]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true)
    
    # 避免除以零，将分母中为零的项替换为一个小值
    denominator = np.where(denominator == 0, 1e-8, denominator)
    
    smape = 100 * np.mean(diff / denominator)
    return smape


def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    
    # 避免除以零，将 y_true 中的零值替换为一个小值
    y_true = np.where(y_true == 0, 1e-8, y_true)
    
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


# 假设 y_test 和 y_pred 已经是标准化反归一化后的数据
# 将其转换为一维数组以确保形状一致
y_test = np.array(y).ravel()
y_pred = np.array(predictions).ravel()

# 评估指标
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = calculate_mape(y_test, y_pred)
smape = calculate_smape(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 打印结果
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')
print(f'R-squared (R²): {r2:.4f}')


Mean Squared Error (MSE): 426.9952
Root Mean Squared Error (RMSE): 20.6639
Mean Absolute Error (MAE): 15.4370
Mean Absolute Percentage Error (MAPE): 18.3681%
Symmetric Mean Absolute Percentage Error (SMAPE): 16.2580%
R-squared (R²): 0.8519


似乎对于较小值预测存在误差

In [31]:
def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    return smape

def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape

## LightGBM 模型预测

In [19]:
# 尝试加载 LightGBM 模型并对标准化后的数据进行预测
import lightgbm as lgb
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# model_path 优先使用文本格式的保存文件，如果没有再尝试 joblib 保存的文件
model_txt = '../result/model/lightgbm_model.txt'
model_pkl = '../result/model/lightgbm_model_joblib.pkl'
model_lgb = None

try:
    # 尝试以 Booster 文本文件加载（如果存在）
    model_lgb = lgb.Booster(model_file=model_txt)
    print('已从', model_txt, '加载 LightGBM 模型')
except Exception as e:
    try:
        # 尝试从 joblib 文件加载
        model_lgb = joblib.load(model_pkl)
        print('已从', model_pkl, '加载 LightGBM 模型（joblib）')
    except Exception as e2:
        print('无法加载 LightGBM 模型:', e, e2)

# 如果 model_lgb 仍为 None，提示用户并停止此单元
if model_lgb is None:
    raise RuntimeError('LightGBM 模型未找到，请先训练并保存模型到 ../result/model/')

# 使用 data_scaled（已标准化的 X）进行预测；确保 data_scaled 已经定义并与训练时列一致
X_for_pred = data_scaled if 'data_scaled' in globals() else X
X_np = X_for_pred.values if hasattr(X_for_pred, 'values') else np.array(X_for_pred)

# 如果模型对象有 best_iteration 属性，优先使用；否则使用全部迭代（num_iteration=-1）
best_iter = getattr(model_lgb, 'best_iteration', None) or -1
y_pred_scaled_lgb = model_lgb.predict(X_np, num_iteration=best_iter)

# 反标准化预测值（使用 scaler_y，如果已加载）
try:
    y_pred_lgb = scaler_y.inverse_transform(y_pred_scaled_lgb.reshape(-1, 1))
except Exception as e:
    print('反标准化失败，返回标准化预测结果，错误:', e)
    y_pred_lgb = y_pred_scaled_lgb.reshape(-1, 1)

# 保存到变量 predictions_lgb，便于后续查看/比较
predictions_lgb = y_pred_lgb

# 显示前几条预测结果
import pandas as pd
df_pred = pd.DataFrame({
    'pred_lgb': predictions_lgb.ravel()
}, index=(X_for_pred.index if hasattr(X_for_pred, 'index') else None))
print(df_pred.head(10))

c:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


已从 ../result/model/lightgbm_model.txt 加载 LightGBM 模型
     pred_lgb
0   57.647768
1  124.299002
2  171.440139
3  161.630560
4  131.966762
5  119.976862
6   95.236133
7   41.831875
8  105.554228
9   70.505281
     pred_lgb
0   57.647768
1  124.299002
2  171.440139
3  161.630560
4  131.966762
5  119.976862
6   95.236133
7   41.831875
8  105.554228
9   70.505281


In [20]:
# 对比前20条真实值和 LightGBM 预测值（如果 y 可用）
if 'y' in globals():
    true_vals = y[:20] if hasattr(y, '__len__') else None
    print('20条 LightGBM 测试结果（真实值 vs 预测值）:')
    for i, pred in enumerate(predictions_lgb[:20]):
        t = true_vals.iloc[i] if hasattr(true_vals, 'iloc') else (true_vals[i] if true_vals is not None else None)
        t_val = t.item() if hasattr(t, 'item') else t
        p_val = pred.item() if hasattr(pred, 'item') else float(pred)
        print(f'第{i+1}条: 真实={t_val}, LightGBM预测={p_val:.2f}')
else:
    print('变量 y 不存在，无法显示真实值对比。')

20条 LightGBM 测试结果（真实值 vs 预测值）:
第1条: 真实=45, LightGBM预测=57.65
第2条: 真实=143, LightGBM预测=124.30
第3条: 真实=154, LightGBM预测=171.44
第4条: 真实=167, LightGBM预测=161.63
第5条: 真实=181, LightGBM预测=131.97
第6条: 真实=138, LightGBM预测=119.98
第7条: 真实=137, LightGBM预测=95.24
第8条: 真实=50, LightGBM预测=41.83
第9条: 真实=136, LightGBM预测=105.55
第10条: 真实=99, LightGBM预测=70.51
第11条: 真实=70, LightGBM预测=67.44
第12条: 真实=62, LightGBM预测=100.83
第13条: 真实=57, LightGBM预测=56.36
第14条: 真实=67, LightGBM预测=51.36
第15条: 真实=167, LightGBM预测=163.45
第16条: 真实=30, LightGBM预测=45.19
第17条: 真实=102, LightGBM预测=80.12
第18条: 真实=163, LightGBM预测=135.51
第19条: 真实=42, LightGBM预测=52.25
第20条: 真实=119, LightGBM预测=82.20


## lightbgm测试指标

In [21]:
# 计算并打印 LightGBM 的评估指标（使用反标准化后的 predictions_lgb 和 y）
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape_local(y_true, y_pred):
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    denominator = np.where(denominator == 0, 1e-8, denominator)
    return 100 * np.mean(np.abs(y_pred - y_true) / denominator)

def calculate_mape_local(y_true, y_pred):
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    y_true = np.where(y_true == 0, 1e-8, y_true)
    return 100 * np.mean(np.abs((y_true - y_pred) / y_true))

if 'y' in globals():
    y_true_all = np.array(y).ravel()
    y_pred_all = np.array(predictions_lgb).ravel()
    mse_l = mean_squared_error(y_true_all, y_pred_all)
    mae_l = mean_absolute_error(y_true_all, y_pred_all)
    rmse_l = np.sqrt(mse_l)
    mape_l = calculate_mape_local(y_true_all, y_pred_all)
    smape_l = calculate_smape_local(y_true_all, y_pred_all)
    r2_l = r2_score(y_true_all, y_pred_all)

    print(f'LightGBM 总体评估 (反标准化后的)：MSE={mse_l:.4f}')
    print(f'LightGBM 总体评估 (反标准化后的)：RMSE={rmse_l:.4f}')
    print(f'LightGBM 总体评估 (反标准化后的)：MAE={mae_l:.4f}')
    print(f'LightGBM 总体评估 (反标准化后的)：MAPE={mape_l:.4f}%')
    print(f'LightGBM 总体评估 (反标准化后的)：SMAPE={smape_l:.4f}%')
    print(f'LightGBM 总体评估 (反标准化后的)：R2={r2_l:.4f}')
else:
    print('变量 y 不存在，无法计算评估指标。')

LightGBM 总体评估 (反标准化后的)：MSE=439.8900
LightGBM 总体评估 (反标准化后的)：RMSE=20.9736
LightGBM 总体评估 (反标准化后的)：MAE=15.6687
LightGBM 总体评估 (反标准化后的)：MAPE=18.2770%
LightGBM 总体评估 (反标准化后的)：SMAPE=16.4245%
LightGBM 总体评估 (反标准化后的)：R2=0.8474


## catboost模型预测

In [29]:
# CatBoost 模型预测：尝试加载模型、预测、反标准化并评估（与 XGBoost/LightGBM 对齐）
try:
    from catboost import CatBoostRegressor
    import joblib
except Exception as e:
    raise RuntimeError('请先安装 catboost: pip install catboost') from e

model_cb_path = '../result/model/catboost_model.cbm'
model_pkl = '../result/model/catboost_model_joblib.pkl'
model_cat = None

# 先尝试使用 CatBoost 的 load_model，再尝试 joblib 回退
try:
    model_cat = CatBoostRegressor()
    model_cat.load_model(model_cb_path)
    print('已从', model_cb_path, '加载 CatBoost 模型')
except Exception as e:
    try:
        model_cat = joblib.load(model_pkl)
        print('已从', model_pkl, '加载 CatBoost 模型（joblib）')
    except Exception as e2:
        print('无法加载 CatBoost 模型:', e, e2)

if model_cat is None:
    raise RuntimeError('CatBoost 模型未找到，请先训练并保存模型到 ../result/model/')

# 准备预测输入（优先使用 data_scaled）
X_for_pred = data_scaled if 'data_scaled' in globals() else X
X_np = X_for_pred.values if hasattr(X_for_pred, 'values') else __import__('numpy').array(X_for_pred)

# 预测（有的 CatBoost 对象要 DataFrame/Pool，也支持 numpy）
try:
    y_pred_scaled_cat = model_cat.predict(X_np)
except Exception:
    try:
        y_pred_scaled_cat = model_cat.predict(X_for_pred)
    except Exception as e:
        raise

# 反标准化（如果使用了 scaler_y）
import numpy as np
try:
    y_pred_cat = scaler_y.inverse_transform(np.array(y_pred_scaled_cat).reshape(-1, 1))
except Exception as e:
    print('反标准化失败，返回标准化预测结果，错误:', e)
    y_pred_cat = np.array(y_pred_scaled_cat).reshape(-1, 1)

# 保存并展示预测结果
predictions_cat = y_pred_cat
import pandas as pd
df_pred_cat = pd.DataFrame({'pred_cat': predictions_cat.ravel()}, index=(X_for_pred.index if hasattr(X_for_pred, 'index') else None))
print(df_pred_cat.head(10))

# 打印前20条真实 vs 预测（如果 y 可用）
if 'y' in globals():
    print('20条 CatBoost 测试结果（真实值 vs 预测值）:')
    for i, pred in enumerate(predictions_cat[:20]):
        t = y.iloc[i] if hasattr(y, 'iloc') else (y[i] if hasattr(y, '__len__') else None)
        t_val = t.item() if hasattr(t, 'item') else t
        p_val = pred.item() if hasattr(pred, 'item') else float(pred)
        print(f'第{i+1}条: 真实={t_val}, CatBoost预测={p_val:.2f}')
else:
    print('变量 y 不存在，无法显示真实值对比。')

# 计算并打印评估指标（如果 y 可用）
if 'y' in globals():
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    import numpy as np

    def calc_smape(a,b):
        a,b = np.array(a).ravel(), np.array(b).ravel()
        denom = (np.abs(a)+np.abs(b))/2.0
        denom = np.where(denom==0, 1e-8, denom)
        return 100*np.mean(np.abs(b-a)/denom)

    def calc_mape(a,b):
        a,b = np.array(a).ravel(), np.array(b).ravel()
        a = np.where(a==0,1e-8,a)
        return 100*np.mean(np.abs((a-b)/a))

    y_true_all = np.array(y).ravel()
    y_pred_all = np.array(predictions_cat).ravel()
    mse_c = mean_squared_error(y_true_all, y_pred_all)
    mae_c = mean_absolute_error(y_true_all, y_pred_all)
    rmse_c = np.sqrt(mse_c)
    mape_c = calc_mape(y_true_all, y_pred_all)
    smape_c = calc_smape(y_true_all, y_pred_all)
    r2_c = r2_score(y_true_all, y_pred_all)

    print(f'CatBoost 总体评估 (反标准化后的)：MSE={mse_c:.4f}')
    print(f'CatBoost 总体评估 (反标准化后的)：RMSE={rmse_c:.4f}')
    print(f'CatBoost 总体评估 (反标准化后的)：MAE={mae_c:.4f}')
    print(f'CatBoost 总体评估 (反标准化后的)：MAPE={mape_c:.4f}%')
    print(f'CatBoost 总体评估 (反标准化后的)：SMAPE={smape_c:.4f}%')
    print(f'CatBoost 总体评估 (反标准化后的)：R2={r2_c:.4f}')

已从 ../result/model/catboost_model.cbm 加载 CatBoost 模型
     pred_cat
0   64.358458
1  135.532126
2  163.524458
3  156.902144
4  134.670682
5  103.481850
6   76.200486
7   46.709479
8  102.326654
9   65.939375
20条 CatBoost 测试结果（真实值 vs 预测值）:
第1条: 真实=45, CatBoost预测=64.36
第2条: 真实=143, CatBoost预测=135.53
第3条: 真实=154, CatBoost预测=163.52
第4条: 真实=167, CatBoost预测=156.90
第5条: 真实=181, CatBoost预测=134.67
第6条: 真实=138, CatBoost预测=103.48
第7条: 真实=137, CatBoost预测=76.20
第8条: 真实=50, CatBoost预测=46.71
第9条: 真实=136, CatBoost预测=102.33
第10条: 真实=99, CatBoost预测=65.94
第11条: 真实=70, CatBoost预测=61.20
第12条: 真实=62, CatBoost预测=87.19
第13条: 真实=57, CatBoost预测=55.42
第14条: 真实=67, CatBoost预测=57.16
第15条: 真实=167, CatBoost预测=158.95
第16条: 真实=30, CatBoost预测=43.17
第17条: 真实=102, CatBoost预测=88.45
第18条: 真实=163, CatBoost预测=90.54
第19条: 真实=42, CatBoost预测=67.46
第20条: 真实=119, CatBoost预测=75.23
CatBoost 总体评估 (反标准化后的)：MSE=485.5559
CatBoost 总体评估 (反标准化后的)：RMSE=22.0353
CatBoost 总体评估 (反标准化后的)：MAE=17.2337
CatBoost 总体评估 (反标准化后的)：MAPE=19.4096%
CatBoost 

## 神经网络融合模型


In [28]:
# 使用已训练的 stacking 神经网络进行预测、评估并保存结果
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_path = '../result/model/stacking_nn_joblib.pkl'
# 1) 加载 stacking 模型（MLP）
try:
    stacking_model = joblib.load(model_path)
    print('已加载 stacking 模型:', model_path)
except Exception as e:
    raise RuntimeError(f'无法加载 stacking 模型：{model_path}，请先训练并保存。错误：{e}')

# 2) 确保基础模型预测存在：predictions (XGBoost), predictions_lgb (LightGBM), predictions_cat (CatBoost)
# 如果任一不存在，尝试从磁盘加载对应模型并计算预测（轻量且带容错）
def ensure_pred(var_name, generator_fn=None):
    if var_name in globals():
        return globals()[var_name]
    if generator_fn is None:
        return None
    try:
        pred = generator_fn()
        globals()[var_name] = pred
        return pred
    except Exception as e:
        print(f'无法生成预测 {var_name}:', e)
        return None

# generator for XGBoost predictions
def gen_xgb():
    import xgboost as xgb
    model_xgb = xgb.Booster()
    model_xgb.load_model('../result/model/guiyihua_xgboost_model_1000.json')
    dmat = xgb.DMatrix(data_scaled)
    pred = model_xgb.predict(dmat)
    try:
        return scaler_y.inverse_transform(pred.reshape(-1,1))
    except Exception:
        return pred.reshape(-1,1)

# generator for LightGBM predictions
def gen_lgb():
    import joblib, lightgbm as lgb, numpy as _np
    model_txt = '../result/model/lightgbm_model.txt'
    model_pkl = '../result/model/lightgbm_model_joblib.pkl'
    try:
        model_lgb = lgb.Booster(model_file=model_txt)
    except Exception:
        model_lgb = joblib.load(model_pkl)
    X_for = data_scaled
    arr = X_for.values if hasattr(X_for, 'values') else _np.array(X_for)
    best_iter = getattr(model_lgb, 'best_iteration', None) or -1
    pred = model_lgb.predict(arr, num_iteration=best_iter)
    try:
        return scaler_y.inverse_transform(pred.reshape(-1,1))
    except Exception:
        return pred.reshape(-1,1)

# generator for CatBoost predictions
def gen_cat():
    import joblib, numpy as _np
    from catboost import CatBoostRegressor
    model_cb = CatBoostRegressor()
    try:
        model_cb.load_model('../result/model/catboost_model.cbm')
    except Exception:
        model_cb = joblib.load('../result/model/catboost_model_joblib.pkl')
    X_for = data_scaled
    arr = X_for.values if hasattr(X_for, 'values') else _np.array(X_for)
    pred = model_cb.predict(arr)
    try:
        return scaler_y.inverse_transform(_np.array(pred).reshape(-1,1))
    except Exception:
        return _np.array(pred).reshape(-1,1)

pred_xgb = ensure_pred('predictions', gen_xgb)
pred_lgb = ensure_pred('predictions_lgb', gen_lgb)
pred_cat = ensure_pred('predictions_cat', gen_cat)

# 3) 检查是否生成成功
if pred_xgb is None or pred_lgb is None or pred_cat is None:
    raise RuntimeError('无法获得所有基础模型的预测，请检查模型文件是否存在或前面的单元是否已运行。')

# 4) 将预测堆叠为元特征 DataFrame，按列顺序 xgb,lgb,cat
meta = pd.DataFrame({'xgb': np.array(pred_xgb).ravel(), 'lgb': np.array(pred_lgb).ravel(), 'cat': np.array(pred_cat).ravel()})
print('meta 特征预览:')
print(meta.head())

# 5) 使用 stacking_model 进行预测（注意：模型可能训练在标准化或原始尺度上；这里假设它预测的是原始尺度）
meta_arr = meta.values
pred_stack = stacking_model.predict(meta_arr)
# 确保为列向量
pred_stack = np.array(pred_stack).reshape(-1,1)

# 6) 诊断：通常 stacking 模型在训练时 target 可能已标准化，预测输出可能是标准化值或原始值
# 我们用一个稳健策略：计算两种候选（原始 pred_stack 与 尝试 inverse_transform(pred_stack)），
# 如果 y 可用，比较两者的 MSE，选择更小的那个；如果 y 不可用，依据数值分布进行简单判定并避免极端值。
def try_inverse(arr):
    try:
        inv = scaler_y.inverse_transform(np.array(arr).reshape(-1,1))
        # 检查是否产生异常值（非常大或包含 NaN）
        if np.isnan(inv).any():
            return None
        if np.isfinite(inv).all() and np.abs(inv).mean() < 1e7:
            return inv
        return None
    except Exception:
        return None

pred_stack_raw = np.array(pred_stack).reshape(-1,1)
pred_stack_inv = try_inverse(pred_stack_raw)
chosen_pred = None
# 如果 y 可用，比较两种预测与真实值的 MSE，并选择更好的那个
if 'y' in globals():
    y_true = np.array(y).ravel()
    # 计算 MSE for raw (if raw is already in original scale, this will be small)
    try:
        mse_raw = mean_squared_error(y_true, pred_stack_raw.ravel())
    except Exception:
        mse_raw = float('inf')
    mse_inv = float('inf')
    if pred_stack_inv is not None:
        try:
            mse_inv = mean_squared_error(y_true, pred_stack_inv.ravel())
        except Exception:
            mse_inv = float('inf')
    # 选择 MSE 更小的预测作为最终输出
    if mse_inv < mse_raw:
        predictions_stack = pred_stack_inv
        print(f'选择了 inverse_transform 后的预测，mse_inv={mse_inv:.4f} < mse_raw={mse_raw:.4f}')
    else:
        predictions_stack = pred_stack_raw
        print(f'选择了原始预测，mse_raw={mse_raw:.4f} <= mse_inv={mse_inv if pred_stack_inv is not None else None}')
    # 计算并打印最终评估指标
    y_pred_vals = np.array(predictions_stack).ravel()
    mse = mean_squared_error(y_true, y_pred_vals)
    mae = mean_absolute_error(y_true, y_pred_vals)
    rmse = np.sqrt(mse)
    def safe_mape(a,b):
        a = np.where(np.array(a)==0, 1e-8, np.array(a))
        return 100*np.mean(np.abs((a-np.array(b))/a))
    mape = safe_mape(y_true, y_pred_vals)
    smape = calculate_smape(y_true, y_pred_vals)
    r2 = r2_score(y_true, y_pred_vals)
    print(f'Stacking NN 总体评估：MSE={mse:.4f}, RMSE={rmse:.4f}, MAE={mae:.4f}, MAPE={mape:.4f}%, SMAPE={smape:.4f}%, R2={r2:.4f}')
    # 同时打印各基模型的评估，便于对比
    def eval_base(name, arr):
        arr = np.array(arr).ravel()
        mse_b = mean_squared_error(y_true, arr)
        rmse_b = np.sqrt(mse_b)
        mae_b = mean_absolute_error(y_true, arr)
        mape_b = safe_mape(y_true, arr)
        smape_b = calculate_smape(y_true, arr)
        r2_b = r2_score(y_true, arr)
        print(f'{name}: MSE={mse_b:.4f}, RMSE={rmse_b:.4f}, MAE={mae_b:.4f}, MAPE={mape_b:.4f}%, SMAPE={smape_b:.4f}%, R2={r2_b:.4f}')
    eval_base('XGBoost', meta['xgb'].values)
    eval_base('LightGBM', meta['lgb'].values)
    eval_base('CatBoost', meta['cat'].values)
else:
    # y 不存在时，选择更可靠的尺度：如果 inverse 后看起来合理（非极端且均值更接近基模型均值），则使用 inverse
    if pred_stack_inv is not None:
        # 简单判定：比较与基模型均值距离
        base_mean = np.mean([meta['xgb'].mean(), meta['lgb'].mean(), meta['cat'].mean()])
        inv_mean = pred_stack_inv.ravel().mean()
        raw_mean = pred_stack_raw.ravel().mean()
        if abs(inv_mean - base_mean) < abs(raw_mean - base_mean):
            predictions_stack = pred_stack_inv
            print('选择 inverse_transform 后的预测（y 不存在，基于均值判定）')
        else:
            predictions_stack = pred_stack_raw
            print('选择原始预测（y 不存在，基于均值判定）')
    else:
        predictions_stack = pred_stack_raw
        print('无法 inverse_transform，选择原始预测（y 不存在）')

# 7) 保存并展示结果
out_df = pd.DataFrame({
    'pred_stack': np.array(predictions_stack).ravel(),
    'pred_xgb': meta['xgb'].values,
    'pred_lgb': meta['lgb'].values,
    'pred_cat': meta['cat'].values
}, index=(data.index if 'data' in globals() else None))
out_path = '../result/model/predictions_stack_from_nn.csv'
out_df.to_csv(out_path, index=True)
print('已保存堆叠预测到', out_path)
print(out_df.head(10))

已加载 stacking 模型: ../result/model/stacking_nn_joblib.pkl
meta 特征预览:
          xgb         lgb         cat
0   62.298950   57.647768   64.358458
1  141.137787  124.299002  135.532126
2  170.305298  171.440139  163.524458
3  169.688278  161.630560  156.902144
4  127.722809  131.966762  134.670682
选择了原始预测，mse_raw=465.4782 <= mse_inv=49632344.36013301
Stacking NN 总体评估：MSE=465.4782, RMSE=21.5749, MAE=16.1262, MAPE=18.6626%, SMAPE=16.8691%, R2=0.8385
XGBoost: MSE=426.9952, RMSE=20.6639, MAE=15.4370, MAPE=18.3681%, SMAPE=16.2580%, R2=0.8519
LightGBM: MSE=439.8900, RMSE=20.9736, MAE=15.6687, MAPE=18.2770%, SMAPE=16.4245%, R2=0.8474
CatBoost: MSE=485.5559, RMSE=22.0353, MAE=17.2337, MAPE=19.4096%, SMAPE=17.7104%, R2=0.8316
选择了原始预测，mse_raw=465.4782 <= mse_inv=49632344.36013301
Stacking NN 总体评估：MSE=465.4782, RMSE=21.5749, MAE=16.1262, MAPE=18.6626%, SMAPE=16.8691%, R2=0.8385
XGBoost: MSE=426.9952, RMSE=20.6639, MAE=15.4370, MAPE=18.3681%, SMAPE=16.2580%, R2=0.8519
LightGBM: MSE=439.8900, RMSE=20.9

In [21]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# import numpy as np

# # 评估模型
# mse = mean_squared_error(y_test, y_pred)
# mae = mean_absolute_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# mape = calculate_mape(y_test, y_pred)
# smape = calculate_smape(y_test, y_pred)

# # 打印结果
# print(f'Mean Squared Error (MSE): {mse:.4f}')
# print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
# print(f'Mean Absolute Error (MAE): {mae:.4f}')
# print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
# print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')

## 保存模型

## 超参数设置

## 不同特征重要程度测试

In [22]:
# import xgboost as xgb
# import matplotlib.pyplot as plt

# # 假设 model 是训练好的 XGBoost 模型
# xgb.plot_importance(model, importance_type='weight', title="Feature Importance (Weight)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='gain', title="Feature Importance (Gain)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='cover', title="Feature Importance (Cover)", height=0.5)
# plt.show()